# Training your first DeepLabCut Model – A step by step example (Part II)

This notebook is used as *part II* in the blogpost **link to blogpost** to train a model in DLC without GPU access. To follow along it is assumed that you already created a DLC project with labeled data in *part I* and uploaded the project to google drive.

## Migrating your DLC project to google colab for GPU access

The great advantage of using Google Colab for cloud computing is that you get free access to a GPU to run your computationaly expensive data analysis. The downsides are that you need to link your google account to use google drive, you may run out of free cloud space and that your project directory paths will be changing every time you move your project to google drive and back.

**Note**: Personally, I wouldn't want to upload the data for the big scientific breackthrough on google drive. But google colab is still a good starting point before investing in expensive GPU upgrades. 

## 1) Colab Environment Configuration 

On the upper left in Google Colab select "**Runtime**", "**change Runtime type**", select "**Python3**" and "**GPU**" as hardware accelerator and click "save". 


Next, install deeplabcut, and at the bottom of the output restart Runtime (this may take a few minutes). 

In [ ]:
!pip install deeplabcut

After restarting runtime, select tensorflow version < 2: 

In [ ]:
%tensorflow_version 1.x

And make sure to import DeeplabCut in light mode, as cloud computing does not allow to use GUIs:

In [2]:
import os
os.environ["DLClight"]="True"

In [ ]:
import deeplabcut

Now you have to mount a valid google drive account to google colab. Follow the link provided to log in to google drive and copy paste the authentication code below:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Now the important part: Because we started the project in our local machine and then exported it to the cloud, all paths are messed up and need to be updated. 

After uploading your DLC project folder to google drive copy the project folder name to reconstruct the new project_path:

In [ ]:
!ls "/content/drive/MyDrive"
ProjectFolderName = 'clock_reding-guillermo-2021-02-15' # Enter the project folder name here as printed below

In [ ]:
project_path = '/content/drive/MyDrive/'+ ProjectFolderName
videofile_path = ['/content/drive/MyDrive/'+ProjectFolderName+'/videos/']
path_config_file = '/content/drive/MyDrive/'+ProjectFolderName+'/config.yaml'

print('NEW project_path: ' + project_path)
print('NEW path_config_file: ' + path_config_file)
print('NEW videofile_path: ' + videofile_path[0])

In [ ]:
# Enter your new project_path below 
!ls '/content/drive/MyDrive/clock_reding-guillermo-2021-02-15'

If you see the content of your DLC project listed in the output above (specially the config.yaml file), copy the new project_path to your project and paste it in the config.yaml file in google drive.

## 2) Training New DLC Model

Create a training dataset from your labeled frames to train your model:

In [ ]:
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

###You are now officially ready to start training! 

Change displayiters to have more or less printed feedback and change saveiters to save the model at intermediate steps. 

**NOTE**: The training will run over several hours, usually over night or even over the weekend. Unfortunately, your google colab session will disconect after a few hours (limitations of free service). Saving the model at intermediate steps will reduce the amount of lost data and will allow you restart the training from the last saved point.

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1, displayiters=100, saveiters=100)

## Evaluate your model

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, 
#so be sure your labels are good! (And you have trained enough ;)

## Restart Training

Typically, you want to train for 50k - 200k iterations. If your colab session terminated before that, run a second training session starting from the last saved iteration.

Find the *train* subdirectory within the dlc-models directory and look for the latest snapshot. Open the pose_cfg.yaml file within the same folder and edit the parameter **init_weights: '<full_path>-snapshot-10000'** without any filetype ending.

When you restart training will be resumed from the las saved pretrained model. 

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1, displayiters=100, saveiters=100)

## Migrating your DLC project back to your local machine

Download the DLC project directory from google drive to your local machine and change the project_directory in the config.yaml file back.

Proceed with jupyter notebook from Part I

In [ ]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='mp4')

In [ ]:
deeplabcut.filterpredictions(path_config_file, videofile_path, videotype='mp4')

In [ ]:
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype='mp4')

In [ ]:
deeplabcut.create_labeled_video(path_config_file, videofile_path, videotype='mp4')

In [ ]:
deeplabcut.export_model(path_config_file)